# Distributed file loading and processing

In this short exercise we will use `DistributedArrays` to load and process four data files in parallel. This can be useful in a scenario where the total data doesn't fit into the memory of a single computer. The idea is to let different workers hold different batches of the data (one file per worker) and load and process the data in parallel on the workers.

(In the following, we will of course only mimic such a situation. Our workers will live on the same machine.)

## Preparation

### Create fake data

Before we turn to the actual tasks, we need some fake data. We will create four binary files holding 1 million random floating point numbers each.

In [24]:
using Serialization
for i in 1:4
    serialize("$i.bin", rand(1000000)) # 8 MB files
end

Let's check if the files have been created (`readdir()` lists all the files and folders of the current directory)

In [27]:
fp = filter(endswith("bin"), readdir())

4-element Array{String,1}:
 "1.bin"
 "2.bin"
 "3.bin"
 "4.bin"

### Start worker processes

Let's get some worker processes which use the same Julia environment as our main session (the environment of the workshop).

In [ ]:
using Distributed, Pkg

In [16]:
withenv("JULIA_PROJECT"=>joinpath(pwd(),"..")) do
    addprocs(4)
end

4-element Array{Int64,1}:
 10
 11
 12
 13

In [28]:
@everywhere using DistributedArrays, Serialization

## Tasks

1. Distribute the array of file names (see the array `fp` above) across the workers. Afterwards, apply the `deserialize` function to load the (fake) data from the four data files in parallel.

2. Convince yourself, that the resulting array, i.e. the loaded data, is distributed equally between the workers (each worker holds the numbers of a single data file).

3. Our goal is to compute the sum of the cosines of all the numbers. First compute the partial sums of the cosines on the workers in parallel (each worker processes the batch of numbers it holds). Afterwards compute the sum of the partial sums on the master.